In [2]:
import sys
sys.path.append("../../")
from src.configs import config

import pandas as pd
import requests
import numpy as np


In [3]:
def convert_to_df(nft_data : dict) -> pd.DataFrame:
    df_data_nft = pd.DataFrame(nft_data.get('nft'), index=[nft_data.get('nft')['token_id']])
    df_data_price_datails = pd.DataFrame(nft_data.get('price_details'), index=[nft_data.get('nft')['token_id']])
    df_data = pd.merge(df_data_nft, df_data_price_datails, left_index=True, right_index=True)
    features = ['marketplace', 'transaction_date', 'seller_address']
    d_features = {}
    for feature in features:
        d_features[feature] = nft_data.get(feature)
    df_data = pd.merge(df_data, pd.DataFrame(d_features, index=[nft_data.get('nft')['token_id']]), left_index=True, right_index=True)
    return df_data


# a function adds new values from transaction data to the dataset
def get_df_from_transaction_data(df_nfts : pd.DataFrame, response : requests.models.Response) -> pd.DataFrame:
    for transaction in response.json()['transactions']:
        df_nfts = pd.concat([df_nfts, convert_to_df(transaction)], ignore_index=True)
    return df_nfts


In [4]:
# Download NFT's data  
# More information: https://docs.nftport.xyz/reference/retrieve-all-transactions

import requests

url = "https://api.nftport.xyz/v0/transactions"

querystring = {"chain":"ethereum","type":"sale"}

headers = {
    "Content-Type": "application/json",
    "Authorization": config.KEY_API
}

# get the first page
response = requests.request("GET", url, headers=headers, params=querystring)
response


<Response [200]>

In [5]:
transaction_features = ['contract_type', 'contract_address', 'token_id', 'asset_type', 'price', 'price_usd', 'marketplace', 'transaction_date']
df_nfts_transaction = get_df_from_transaction_data(pd.DataFrame(columns=transaction_features), response)

In [6]:
df_nfts_transaction.head()

,contract_type,contract_address,token_id,asset_type,price,price_usd,marketplace,transaction_date,seller_address,contract_address_x,contract_address_y
0,ERC721,0x395fc9db003e58b804188155dc7e0e609a062f61,2809,ETH,0.019,26.952512,opensea,2022-09-18T16:31:47,0x2a8c653439f4e479089ccaca69b025279ac1a415,NaN,NaN
1,ERC721,0xb39f5f5199042086d6f162ededb761d0f33a2849,1257,ETH,0.022,31.208171,opensea,2022-09-18T16:31:47,0xfcba4dad18dc334f97e000105653ac7347294c0f,NaN,NaN
2,ERC721,0x8270fc3b2d23de703b265b2abe008883954fea8e,1268,ETH,0.220,312.081714,opensea,2022-09-18T16:31:47,0x8bc45e6d75aac519d8574f67d709e3a1ae21f3b7,NaN,NaN
3,ERC721,NaN,8815044505892096730262426646285359585117131684...,ERC20,0.000,0.000000,opensea,2022-09-18T16:31:59,0xc1487f151e0f02ac227defd5eca5dbba82c4c047,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,ERC721,0x2c88aa0956bc9813505d73575f653f69ada60923,93,ETH,0.000,0.000000,opensea,2022-09-18T16:31:59,0xd102753b2f83f7b46027483360227afab13ff48d,NaN,NaN


In [15]:
response.json()

{'response': 'OK',
 'transactions': [{'type': 'sale',
   'buyer_address': '0xbf05cca381a8ad0177fb967341d040c6fa980335',
   'seller_address': '0x6b6988334e0236aeec69f78127579a84bb06bc3d',
   'nft': {'contract_type': 'ERC721',
    'contract_address': '0xe9814ccd783a12ffc65e447c436b51a8a167fafd',
    'token_id': '2422'},
   'quantity': 1,
   'price_details': {'asset_type': 'ETH',
    'price': 0.025,
    'price_usd': 36.45861853275912},
   'transaction_hash': '0xbdb731c70ffb11dc45899485c4797fba55ed015248a4248ae549bc3a4dcbb122',
   'block_hash': '0x561f1956f98915147aab5e461cbf185febff368dfffeb0f67d01421cbbc18113',
   'block_number': 15558651,
   'transaction_date': '2022-09-18T06:28:59'},
  {'type': 'sale',
   'buyer_address': '0x56dd5bbede9bfdb10a2845c4d70d4a2950163044',
   'seller_address': '0x438b6dec029819e0d8be740d9b9064a9bedbf84c',
   'nft': {'contract_type': 'ERC721',
    'contract_address': '0x9c62e0457b89fdce6ced21f8363439c630466473',
    'token_id': '374'},
   'quantity': 1,
   'p

In [16]:
import time
# Get information from the next pages
url = "https://api.nftport.xyz/v0/transactions"

# each page gives us 50 more nfts
num_pages = 100

for _ in range(num_pages):
    querystring = {
        "chain":"ethereum",
        "type":"sale",
        "continuation": response.json()['continuation']
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": config.KEY_API,
    }

    # get the first page
    # time.sleep(0.05)
    response = requests.request("GET", url, headers=headers, params=querystring)
    print(response)
    if response.status_code == 200:
        df_nfts_transaction = get_df_from_transaction_data(df_nfts_transaction, response)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [17]:
df_nfts_transaction

,contract_type,contract_address,token_id,asset_type,price,price_usd,marketplace,transaction_date,seller_address,contract_address_x,contract_address_y
0,ERC721,0x1d6b7452f381e85ad0c04ffd018150eb54e73f8e,198,ETH,0.0000,0.000000,opensea,2022-09-18T06:25:11,0xd9a02db049e1d875bf9e72ecc607dea6d8d4edea,NaN,NaN
1,ERC721,0x2d004b72d8b7d36f9da2e4a14516618bf53bac57,2170074014104906318993168896,ETH,0.0000,0.000000,opensea,2022-09-18T06:25:11,0x5438f01ebba17c7ee279af990d0a8a02267833e4,NaN,NaN
2,ERC721,0x9378368ba6b85c1fba5b131b530f5f5bedf21a18,23021,ETH,0.0000,0.000000,opensea,2022-09-18T06:25:11,0x9c02f81016136d69656ddec74f7bec54413de1c0,NaN,NaN
3,ERC721,0x9378368ba6b85c1fba5b131b530f5f5bedf21a18,23021,ETH,0.0000,0.000000,opensea,2022-09-18T06:25:11,0x9c02f81016136d69656ddec74f7bec54413de1c0,NaN,NaN
4,ERC721,0x9378368ba6b85c1fba5b131b530f5f5bedf21a18,23021,ETH,0.0000,0.000000,opensea,2022-09-18T06:25:11,0x9c02f81016136d69656ddec74f7bec54413de1c0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5045,ERC721,0xd6f4a923e2ecd9ab7391840ac78d04bfe40bd5e1,1126,ETH,0.1860,270.674982,opensea,2022-09-18T07:59:47,0xdb2ba8fc10bf858111a069e0614c10a4ce339ac1,NaN,NaN
5046,ERC721,0xd6f4a923e2ecd9ab7391840ac78d04bfe40bd5e1,1029,ETH,0.2400,349.258041,opensea,2022-09-18T07:59:47,0x16679457e94eee1aee98be91c8811fd05a523698,NaN,NaN
5047,ERC721,0x4be08b9db3eb7690b3d0362fde239aec919483d0,5088,ETH,0.0044,6.403064,opensea,2022-09-18T07:59:47,0x01b44b1018c0629ead48fc88c59d56f3894b1535,NaN,NaN
5048,ERC721,0xe62482263ac31d229875dcb9e5cfdadd7627e495,8571,ETH,0.0040,5.820967,opensea,2022-09-18T07:59:47,0xaa8d1d1a31da6d3f5e68e66ae03c8139d2adfb0e,NaN,NaN


In [18]:
# a funtction adds new values from metadata to the dataset
def add_nfts(df_nfts: pd.DataFrame, response: requests.models.Response) -> pd.DataFrame:
    nft = response.json().get('nft')
    if nft == None:
        return df_nfts
    row = np.array([nft.get(el) for el in df_nfts.keys()])
    df_nfts = pd.concat([df_nfts, pd.DataFrame([row], columns = df_nfts.keys())], ignore_index=True)
    return df_nfts


def get_attributes(sr_attributes: pd.Series, response: requests.models.Response) -> pd.Series:
    nft = response.json().get('nft')
    if nft == None or nft.get('metadata') == None:
        return sr_attributes
    sr_attributes = pd.concat(
        [sr_attributes, 
        pd.Series(  
            [nft.get('metadata').get('attributes')], 
            name='attributes',
            index=[nft.get('token_id')])
        ])
    return sr_attributes

In [20]:
import time
features = ['token_id', 'chain', 'contract_address']
df_nfts_contract = pd.DataFrame(columns = features)
sr_attributes = pd.Series(dtype='object', name='attributes')
for irow in df_nfts_transaction.index:
    contract = df_nfts_transaction['contract_address'][irow]
    token = df_nfts_transaction['token_id'][irow]
    url = f"https://api.nftport.xyz/v0/nfts/{contract}/{token}"
    # time.sleep(0.05) # we have to wait some time due to the limitation of free access
    querystring = {"chain":"ethereum"}

    headers = {
        "Content-Type": "application/json",
        "Authorization": config.KEY_API
    }

    nft_response = requests.request("GET", url, headers=headers, params=querystring)
    print(nft_response.status_code)
    if nft_response.status_code == 200:
        df_nfts_contract = add_nfts(df_nfts_contract, nft_response)
        sr_attributes = get_attributes(sr_attributes, nft_response)
    else:
        print(nft_response.text)


200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits

200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits

429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
404
{"response":"NOK","error":{"status_code":404,"code":"not_found","message":"The requested NFT was not found in our database."}}
422
{"response":"NOK","error":{"status_code":422,"code":"invalid_address","message":"Invalid address: contract_address."}}
200
429
{"error":{"status_code":429,"code":"too_many_requests","mess

429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https:

429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https:

...

200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits

200
200
200
200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
422
{"response":"NOK","error":{"status_code":422,"code":"invalid_address","message":"Invalid address: contract_address."}}
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff o

200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits

200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits

200
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
422
{"response":"NOK","error":{"status_code":422,"code":"invalid_address","message":"Invalid address: contract_address."}}
200
200
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your requests. See rate limit details at https://docs.nftport.xyz/docs/nftport/ZG9jOjQ1ODQ4NzQ4-quotas. If you need higher limits, see https://www.nftport.xyz/pricing"}}
429
{"error":{"status_code":429,"code":"too_many_requests","message":"Too many requests hit the API too quickly (per one second). We recommend an exponential backoff of your reque

In [22]:
df_nfts_transaction_1 = df_nfts_transaction.drop(['contract_address_x', 'contract_address_y'], axis=1)
df_nfts_transaction_1.drop_duplicates(subset=['token_id'], inplace=True)

In [23]:
df_nfts_contract_1 = df_nfts_contract.drop(['contract_address'], axis=1)
df_nfts_contract_1.drop_duplicates(subset=['token_id'], inplace=True)


In [24]:
df_nfts_contract_with_attr = pd.merge(df_nfts_contract_1, sr_attributes, how='left', left_on='token_id', right_index=True)
df_nfts_contract_1.head(15)

,token_id,chain
0,198,ethereum
1,2170074014104906318993168896,ethereum
2,23021,ethereum
4,5353,ethereum
5,2977,ethereum
6,5238,ethereum
7,6373,ethereum
8,4714,ethereum
9,6413,ethereum
10,880,ethereum


In [25]:
# df_nfts_contract_with_attr = pd.merge(df_nfts_contract_1, sr_attributes, left_on=['token_id'], right_index=True)
df_final = pd.merge(df_nfts_contract_with_attr, df_nfts_transaction_1,  how='left', on='token_id')
df_final.drop_duplicates(subset=['token_id'], inplace=True)
df_final.drop(['contract_type', 'chain', 'token_id', 'contract_address', 'asset_type', 'marketplace'], axis=1, inplace=True)
df_final.rename(columns={'price' : 'price_eth'}, inplace=True)
df_final.dropna(subset=['attributes'], inplace=True)
df_final

,attributes,price_eth,price_usd,transaction_date,seller_address
0,"[{'trait_type': 'artist', 'value': 'aleqth'}, ...",0.0000,0.000000,2022-09-18T06:25:11,0xd9a02db049e1d875bf9e72ecc607dea6d8d4edea
2,"[{'trait_type': 'Level', 'value': 84}, {'trait...",0.0000,0.000000,2022-09-18T06:25:11,0x5438f01ebba17c7ee279af990d0a8a02267833e4
3,"[{'trait_type': 'Character', 'value': 'Amiable...",0.0000,0.000000,2022-09-18T06:25:11,0x9c02f81016136d69656ddec74f7bec54413de1c0
5,"[{'trait_type': 'Background', 'value': 'Light ...",0.0000,0.000000,2022-09-18T06:25:11,0x68d999feb381e991a389a0839bc53b70046cb60d
6,"[{'trait_type': 'type', 'value': 'female'}, {'...",0.0000,0.000000,2022-09-18T06:25:11,0x85d50139a55d61748845736b859c882baa8cbcdb
...,...,...,...,...,...
3129,"[{'trait_type': 'Edition', 'value': 'Open Edit...",0.0004,0.582097,2022-09-18T07:59:47,0xc449ab8cfb39053f27046a61868c70925b840e11
3130,"[{'trait_type': 'Background', 'value': 'AB Emb...",0.1870,272.130224,2022-09-18T07:59:47,0x6132b62ed5b80b842e9dc1b7999d0a8f9cf51441
3131,"[{'trait_type': 'Background', 'value': 'Sunset...",0.1300,189.181439,2022-09-18T07:59:47,0xf3cb88465f7ff4cb5b509a4fde91d8132c5c1ebd
3132,"[{'trait_type': 'BACK GROUND COLOR', 'value': ...",0.0450,65.485883,2022-09-18T07:59:47,0x44b059ecae0ddd09b088c787645b6342590e0836


In [26]:
def convert_attributes(traits):
    res = dict()
    dict_synonyms = {'clothing' : 'clothes', 
                    'eye' : 'eyes',
                    }
    try:
        for trait in traits:
            if trait['trait_type'].lower() in dict_synonyms.keys():
                res[dict_synonyms[trait['trait_type'].lower()]] = trait.get('value')
            elif trait['trait_type'].lower() == 'eye':
                res['eyes'] = trait.get('value')
            else:
                res[trait['trait_type'].lower()] = trait.get('value')
        return res
    except (TypeError, AttributeError, KeyError):
        return None


In [27]:
df_final_1 = df_final.copy()
df_final_1['attributes'] = df_final['attributes'].apply(convert_attributes)
df_final_1


,attributes,price_eth,price_usd,transaction_date,seller_address
0,"{'artist': 'aleqth', 'style': 'penandmarker'}",0.0000,0.000000,2022-09-18T06:25:11,0xd9a02db049e1d875bf9e72ecc607dea6d8d4edea
2,"{'level': 84, 'zone': 'Battle', 'rare genes bo...",0.0000,0.000000,2022-09-18T06:25:11,0x5438f01ebba17c7ee279af990d0a8a02267833e4
3,"{'character': 'Amiable Anchovy', 'pose': 'Rest...",0.0000,0.000000,2022-09-18T06:25:11,0x9c02f81016136d69656ddec74f7bec54413de1c0
5,"{'background': 'Light Blue', 'body': 'Bored Mo...",0.0000,0.000000,2022-09-18T06:25:11,0x68d999feb381e991a389a0839bc53b70046cb60d
6,"{'type': 'female', 'skin': 'pink', 'outfit': '...",0.0000,0.000000,2022-09-18T06:25:11,0x85d50139a55d61748845736b859c882baa8cbcdb
...,...,...,...,...,...
3129,{'edition': 'Open Edition'},0.0004,0.582097,2022-09-18T07:59:47,0xc449ab8cfb39053f27046a61868c70925b840e11
3130,"{'background': 'AB Emblem & Script on Red', 'c...",0.1870,272.130224,2022-09-18T07:59:47,0x6132b62ed5b80b842e9dc1b7999d0a8f9cf51441
3131,"{'background': 'Sunset', 'fur': 'Dalmatian fur...",0.1300,189.181439,2022-09-18T07:59:47,0xf3cb88465f7ff4cb5b509a4fde91d8132c5c1ebd
3132,"{'back ground color': 'Green', 'back ground pa...",0.0450,65.485883,2022-09-18T07:59:47,0x44b059ecae0ddd09b088c787645b6342590e0836


In [28]:
dict_traits = dict()
for attr in df_final_1.attributes:
    if attr:
        try:
            for trait in attr:
                dict_traits[trait] = dict_traits.setdefault(trait, 0) + 1
        except AttributeError:
            pass
        except TypeError:
            # print(trait)
            pass
        
preparred_for_df = []
for key in dict_traits:
    preparred_for_df.append([key, dict_traits[key]])
pd.DataFrame(preparred_for_df, columns=['trait_type','quantity']).sort_values(by=['quantity'], ascending=False).head(20)

,trait_type,quantity
22,background,1145
14,eyes,657
15,mouth,577
59,shoes,510
56,skeleton,502
57,socks,500
58,bottoms,436
60,tops,382
5,edition,332
29,clothes,314


In [29]:
attributes = ['background', 'eyes', 'body', 'mouth', 'head', 'clothes', 'hair']
df_separated_attributes = df_final_1.copy()


for attr in attributes:
    df_separated_attributes[attr] = df_final_1['attributes'].apply(lambda x: x.get(attr).lower() if x and type(x.get(attr)) is str else None)
df_separated_attributes.drop(['attributes'], axis=1, inplace=True)
df_separated_attributes

,price_eth,price_usd,transaction_date,seller_address,background,eyes,body,mouth,head,clothes,hair
0,0.0000,0.000000,2022-09-18T06:25:11,0xd9a02db049e1d875bf9e72ecc607dea6d8d4edea,None,None,None,None,None,None,None
2,0.0000,0.000000,2022-09-18T06:25:11,0x5438f01ebba17c7ee279af990d0a8a02267833e4,None,spider (mythic),cow,pipe,None,None,None
3,0.0000,0.000000,2022-09-18T06:25:11,0x9c02f81016136d69656ddec74f7bec54413de1c0,None,None,None,None,None,None,None
5,0.0000,0.000000,2022-09-18T06:25:11,0x68d999feb381e991a389a0839bc53b70046cb60d,light blue,closed,bored motor corp 14,pizza,None,None,None
6,0.0000,0.000000,2022-09-18T06:25:11,0x85d50139a55d61748845736b859c882baa8cbcdb,None,scientist,None,meh,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
3129,0.0004,0.582097,2022-09-18T07:59:47,0xc449ab8cfb39053f27046a61868c70925b840e11,None,None,None,None,None,None,None
3130,0.1870,272.130224,2022-09-18T07:59:47,0x6132b62ed5b80b842e9dc1b7999d0a8f9cf51441,ab emblem & script on red,None,None,None,None,None,None
3131,0.1300,189.181439,2022-09-18T07:59:47,0xf3cb88465f7ff4cb5b509a4fde91d8132c5c1ebd,sunset,makeup,None,grills,None,None,None
3132,0.0450,65.485883,2022-09-18T07:59:47,0x44b059ecae0ddd09b088c787645b6342590e0836,None,eye purple2,body p1,None,None,tasukigake green,red3


In [174]:
df_separated_attributes.fillna(value=np.nan, inplace=True)

In [30]:
df_separated_attributes.dropna(subset=['background'])


,price_eth,price_usd,transaction_date,seller_address,background,eyes,body,mouth,head,clothes,hair
5,0.0000,0.000000,2022-09-18T06:25:11,0x68d999feb381e991a389a0839bc53b70046cb60d,light blue,closed,bored motor corp 14,pizza,None,None,None
7,0.0020,2.916689,2022-09-18T06:25:11,0xb366c83f5a37494727efc1d1c9dd8eab068487cd,red,closed,None,bored,hippie hair,gym tee,None
21,0.0299,43.604508,2022-09-18T06:25:23,0xd5c544e7fd5e2b6b87feb962cc1e61a6f85db070,cool gray,tired,None,long stubble,None,straw poncho,brown long
43,0.0000,0.000000,2022-09-18T06:25:35,0x97a782b3c06cc2e86a33c8b4e1a6402aa51d5e29,crystal playground,blackout,None,shadow,shadow,None,None
47,0.0000,0.000000,2022-09-18T06:25:47,0xe8b8febd75aadb85df33ade6e7fa261093959c5d,blue_background,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
3118,0.0190,27.649595,2022-09-18T07:59:11,0xe52af5edaa005901acf4509e3b7bfb7d2b431bf3,blue_background,None,None,None,None,None,None
3119,0.0030,4.365726,2022-09-18T07:59:11,0x8a0a677d64e5a428bdd04c2a96610010835396f6,katsuobushi,None,None,None,None,None,None
3130,0.1870,272.130224,2022-09-18T07:59:47,0x6132b62ed5b80b842e9dc1b7999d0a8f9cf51441,ab emblem & script on red,None,None,None,None,None,None
3131,0.1300,189.181439,2022-09-18T07:59:47,0xf3cb88465f7ff4cb5b509a4fde91d8132c5c1ebd,sunset,makeup,None,grills,None,None,None


In [31]:
df_separated_attributes.to_csv('Dataset_0.5_yb.csv', index=False)